In [1]:
# Basic lib imports
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

In [79]:
#AAL atlas paths
AAL_LEFT = '/opt/quarantine/CIVET/1.1.12/build/CIVET-1.1.12/models/AAL_atlas_left.txt'
AAL_RIGHT = '/opt/quarantine/CIVET/1.1.12/build/CIVET-1.1.12/models/AAL_atlas_right.txt'
AAL_roi_names = '/home/cic/bhanik/Downloads/AAL_ROI_names_number_map.txt'

AAL_lowres_threshold = 40962

In [69]:
# Return mean thickness per ROI for a given subject-vertex data
def apply_atlas(atlas_data,sub_data):
    assert len(atlas_data) == len(sub_data)
    label_set = np.unique(atlas_data)
    
    # mask subject data with atlas
    sub_roi_dict = {}
    for label in label_set:
        sub_roi_dist = sub_data[atlas_data==label]
        sub_roi_dict[label] = np.mean(sub_roi_dist)
        
    return sub_roi_dict

def get_cohort_data(subject_dir,atlas_data):
    label_set = np.unique(atlas_data)
    print('# of unique labels: {}, max label: {}, min label: {}'.format(len(label_set),np.max(atlas_data),np.min(atlas_data)))

    subject_ids = os.listdir(subject_dir)
    print('Number of subjects: {}'.format(len(subject_ids)))

    cohort_df = pd.DataFrame(columns=['subject_id']+list(label_set))
    for sub_id in subject_ids:
        sub_file = '{}/thickness/nih_chp_{}_native_rms_rsl_tlink_30mm_left.txt'.format(sub_id,sub_id)
        sub_data = pd.read_csv(subject_dir + sub_file,header=None).values
        sub_roi_dict = apply_atlas(atlas_data,sub_data)
        sub_df =  pd.DataFrame([sub_roi_dict], columns=list(sub_roi_dict.keys()))
        sub_df['subject_id'] = [sub_id]
        cohort_df = cohort_df.append(sub_df,ignore_index=True)
    
    print('Dataframe size {}'.format(cohort_df.shape))
    return cohort_df


In [95]:
# Read atlas file   
atlas_data_L = pd.read_csv(AAL_LEFT,header=None).values[:AAL_lowres_threshold]
atlas_data_R = pd.read_csv(AAL_RIGHT,header=None).values[:AAL_lowres_threshold]

# Read subject file
subject_dir = '/data/chamal/projects/nikhil/data/ASD_ML/data/armin_data/civet_out/mallar_all/'
subject_aal_L = get_cohort_data(subject_dir,atlas_data_L)
subject_aal_R = get_cohort_data(subject_dir,atlas_data_R)

subject_data = pd.merge(subject_aal_L,subject_aal_R,on='subject_id')


# of unique labels: 40, max label: 89, min label: 0
Number of subjects: 1535


/data/chamal/projects/nikhil/code/python/conda/anaconda2/envs/tensorflow_v14/lib/python3.5/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Dataframe size (1535, 41)
# of unique labels: 40, max label: 90, min label: 0
Number of subjects: 1535
Dataframe size (1535, 41)


In [96]:
# Map the ROI Ids and names
aal_roi_names = pd.read_csv(AAL_roi_names,delimiter='\'',header=None)
aal_roi_names = aal_roi_names[[0,1,3]]
aal_roi_names.columns = ['id','roi','name']
aal_roi_names_dict = dict(zip(aal_roi_names['id'], aal_roi_names['roi']))
aal_roi_names_dict['0_x'] = 'background_L'
aal_roi_names_dict['0_y'] = 'background_R'
subject_data = subject_data.rename(columns=aal_roi_names_dict)
subject_data.to_csv('/data/chamal/projects/nikhil/data/ASD_ML/data/armin_data/csv/vertex_to_aal.csv')

In [87]:
aal_roi_names

,id,roi,name
0,27,REC.L,Left Gyrus Rectus
1,21,OLF.L,Left Olfactory Cortex
2,5,ORBsup.L,"Left Supeiror frontal gyrus, orbital part"
3,25,ORBsupmed.L,"Left Superior frontal gyrus, medial orbital"
4,9,ORBmid.L,Left Middle frontal gyrus orbital part
5,15,ORBinf.L,"Left Inferior frontal gyrus, orbital part"
6,3,SFGdor.L,"Left Superior frontal gyrus, dorsolateral"
7,7,MFG.L,Left Middle frontal gyrus
8,11,IFGoperc.L,"Left Inferior frontal gyrus, opercular part"
9,13,IFGtriang.L,"Left Inferior frontal gyrus, triangular part"
